In [ ]:
# This is my first submission, I barely know what i'm doing, so... that's it.

In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
from decimal import Decimal
from IPython.core.display import display, HTML
pd.options.mode.chained_assignment = None
import matplotlib.pyplot as plt # Visualização dos dados
from scipy import stats
from sklearn import model_selection
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import confusion_matrix
display(HTML("<style>.container { width:70% !important; }</style>"))
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

In [ ]:
dados_originais = pd.read_csv('../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')

In [ ]:
dados_originais[dados_originais['gender'] == 'Other']

In [ ]:
dados_originais = dados_originais[dados_originais['gender'] != "Other"] # Removing gender because there's only 1 row 

In [ ]:
dados = dados_originais.copy()

In [ ]:
dados.rename(columns={
    'id': 'Id',
    'gender': 'Gênero',
    'age': 'Idade',
    'hypertension': 'Hipertenso',
    'heart_disease': 'Doença cardíaca',
    'ever_married': 'Já foi casado',
    'work_type': 'Tipo de trabalho',
    'Residence_type': 'Tipo de residência',
    'avg_glucose_level': 'Nivel médio de glicose',
    'bmi': 'IMC',
    'smoking_status': 'Cigarro',
    'stroke': 'AVC'
    }, inplace=True)

In [ ]:
dados["Gênero"] = dados['Gênero'].replace("Male", "Homem").replace("Female", "Mulher")
dados['Idade'] = dados['Idade'].astype(int)
dados['Já foi casado'] = dados['Já foi casado'].replace("Yes", "Sim").replace("No", "Não")
dados['Cigarro'] = (dados['Cigarro'].replace
    ("formerly smoked", "Já fumou").replace
    ("never smoked", "Nunca fumou").replace
    ("smokes", "Fuma").replace
    ("Unknown", "Não informado"))
dados['Tipo de trabalho'] = (dados['Tipo de trabalho'].replace
                                  ("Private", "Empresa privada").replace
                                  ("Self-employed", "Autônomo").replace
                                  ("Govt_job", "Funcionário público").replace
                                  ("Never_worked", "Nunca trabalhou").replace
                                  ("children", "Menor de idade"))
dados['Tipo de residência'] = (dados['Tipo de residência'].replace
                               ("Urban", "Urbana").replace
                               ("Rural", "Rural"))
dados['Hipertenso'] = (dados['Hipertenso'].replace
                            (0, "Não").replace
                            (1, "Sim"))
dados['AVC'] = (dados['AVC'].replace
                (0, "Não").replace
                (1, "Sim"))

In [ ]:
dados['IMC'] = round(dados['IMC'].fillna(np.mean(dados['IMC'])),2)
dados['Nivel médio de glicose'] = round(dados['Nivel médio de glicose'].fillna(np.mean(dados['Nivel médio de glicose'])),2)

### Criando coluna para Faixa etária

In [ ]:
faixa_etaria = []
for x in dados['Idade']:
    if x > 0 and x <= 20:
        faixa_etaria.append("Até 20 anos")
    elif x > 20 and x <= 40:
        faixa_etaria.append("21 a 40 anos")
    elif x > 40 and x <= 60:
        faixa_etaria.append("41 a 60 anos")
    elif x > 60 and x <= 80:
        faixa_etaria.append("61 a 80 anos")
    elif x > 80 and x <= 90:
        faixa_etaria.append("81 a 90 anos")
    else:
        faixa_etaria.append("91 anos ou mais")
dados['Faixa etária'] = faixa_etaria

### Criando coluna para faixa de peso de acordo com o IMC

In [ ]:
faixa_peso = []
peso = []
for x in dados['IMC']:
    if x <= 17:
        faixa_peso.append("Muito abaixo do peso")
        peso.append(0)
    elif x >= 17 and x <= 18.49:
        faixa_peso.append("Abaixo do peso")
        peso.append(1)
    elif x >= 18.50 and x <= 24.99:
        faixa_peso.append("Peso normal")
        peso.append(2)
    elif x >= 25 and x <= 29.99:
        faixa_peso.append("Acima do peso")
        peso.append(3)
    elif x >= 30 and x <= 34.99:
        faixa_peso.append("Obesidade I")
        peso.append(4)
    elif x >= 35 and x <= 39.99:
        faixa_peso.append("Obesidade II (severa)")
        peso.append(5)
    elif x >= 40:
        faixa_peso.append("Obesidade III (mórbida)")
        peso.append(6)
dados['Índice IMC'] = faixa_peso

### Criando coluna para nível de glicose

In [ ]:
nivel_glicose = []
for x in dados['Nivel médio de glicose']:
    if x <= 99:
        nivel_glicose.append("Normal")
    elif x > 99 and x <= 125:
        nivel_glicose.append("Pré-diabetes")
    elif x > 125:
        nivel_glicose.append("Diabetes")
dados['Diabetes'] = nivel_glicose

<h2><center>Visualização por Gênero x Idade</center></h2>

In [ ]:
sns.displot(data=dados[dados['AVC'] == 'Sim'], x='Idade', hue='Gênero', col='AVC', kde=True, multiple='stack', col_wrap=1);
print("Porcentagem de pessoas que sofreram AVC de acordo com o gênero:")
dados[dados['AVC'] == 'Sim']['Faixa etária'].value_counts(normalize=True).mul(100).round(2).astype(str) + '%'

<h2><center>Visualização de pessoas que sofreram AVC pela relação com o cigarro | AVC = Sim</center></h2>

In [ ]:
sns.displot(data=dados[dados['AVC'] == 'Sim'], x='Idade', hue='AVC', col='Cigarro', kde=True, multiple='stack', col_wrap=2);
print("Porcentagem de pessoas que sofreram AVC em relação ao cigarro:")
dados[dados['AVC'] == 'Sim']['Cigarro'].value_counts(normalize=True).mul(100).round(2).astype(str) + '%'

<h2><center>Visualização por Tipo de Trabalho, Idade e Gênero</center></h2>

In [ ]:
sns.displot(data=dados[dados['AVC'] == 'Sim'], x='Idade', hue='Tipo de trabalho', col='AVC', kde=True, multiple='stack', col_wrap=1);
print("Porcentagem de pessoas que sofreram AVC por tipo de trabalho:")
dados[dados['AVC'] == 'Sim']['Tipo de trabalho'].value_counts(normalize=True).mul(100).round(2).astype(str) + '%'

<h2><center>Visualização por Idade x Peso | AVC = Sim</center></h2>

In [ ]:
sns.displot(data=dados[dados['AVC'] == 'Sim'], x='Idade', col='Índice IMC', kde=True, multiple='stack', col_wrap=3);
dados[dados['AVC'] == 'Sim']['Índice IMC'].value_counts(normalize=True).mul(100).round(2).astype(str) + '%'

In [ ]:
dados_pairplot = dados.drop(columns='Id')

<h2><center>Correlação das variáveis | AVC = Sim</center></h2>

In [ ]:
g = sns.pairplot(dados_pairplot[dados_pairplot['AVC'] == "Sim"], hue="Gênero", palette="Set2", diag_kind="kde", height=2.5)

In [ ]:
dados_pred = dados.copy()

In [ ]:
dados_pred.head()

In [ ]:
dados_pred["Gênero"] = dados_pred['Gênero'].replace("Homem", 0).replace("Mulher", 1)
dados_pred['Já foi casado'] = dados_pred['Já foi casado'].replace("Sim", 1).replace("Não", 0)
dados_pred['Cigarro'] = (dados_pred['Cigarro'].replace
    ("Já fumou", 0).replace
    ("Nunca fumou", 1).replace
    ("Fuma", 2).replace
    ("Não informado", 3))
dados_pred['Tipo de trabalho'] = (dados_pred['Tipo de trabalho'].replace
                                  ("Empresa privada", 0).replace
                                  ("Autônomo", 1).replace
                                  ("Funcionário público", 2).replace
                                  ("Nunca trabalhou", 3).replace
                                  ("Menor de idade", 4))
dados_pred['Tipo de residência'] = (dados_pred['Tipo de residência'].replace
                               ("Urbana", 0).replace
                               ("Rural", 1))
dados_pred['AVC'] = (dados_pred['AVC'].replace
                     ("Não", 0).replace
                     ("Sim", 1)
                    )
dados_pred['Hipertenso'] = (dados_pred['Hipertenso'].replace
                               ("Não", 0).replace
                               ("Sim", 1))
dados_pred['Idade'] = dados_pred['Idade'].astype(int)
dados_pred['IMC'] = peso

In [ ]:
idade = []
for x in dados_pred['Idade']:
    if x > 0 and x <= 20:
        idade.append(0) # Até 20 anos
    elif x > 20 and x <= 40:
        idade.append(1) # 21 a 40 anos
    elif x > 40 and x <= 60:
        idade.append(2) # 41 a 60 anos
    elif x > 60 and x <= 80:
        idade.append(3) # 61 a 80 anos
    elif x > 80 and x <= 90:
        idade.append(4) # 81 a 90 anos
    else:
        idade.append(5) # 91 anos ou mais
dados_pred['Faixa etária'] = idade

In [ ]:
glicose = []
for x in dados_pred['Nivel médio de glicose']:
    if x <= 99:
        glicose.append(0)
    elif x > 99 and x <= 125:
        glicose.append(1)
    elif x > 125:
        glicose.append(2)
dados_pred['Diabetes'] = glicose

In [ ]:
dados_pred.describe()

In [ ]:
dados_pred.drop(columns=['Índice IMC', 'Nivel médio de glicose', 'Idade'], inplace=True)

In [ ]:
dados_pred

In [ ]:
X = dados_pred.drop(columns=['Id', 'AVC'])
y = dados_pred['AVC']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)

In [ ]:
from sklearn.naive_bayes import GaussianNB

# Inicializar nosso classificador
gnb = GaussianNB()

# Treinar nosso classificador
model = gnb.fit(X_train, y_train)

preds = gnb.predict(X_test)
print(preds)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, preds, average='micro')

In [ ]:
df_predicao = pd.DataFrame({"Valor real": y_test,
                            "Valor predito": preds})

In [ ]:
df_submit = pd.merge(dados_pred['Id'], df_predicao['Valor predito'], left_index=True, right_index=True)

In [ ]:
df_submit.to_csv("Kaggle_stroke.csv")